In [ ]:
import os
try:
    import google.colab
    os.system('pip install problog sorcery pygrove datasets')
except:
    pass
bucket=''
usnli_path=f's3://{bucket}/usnli'
output_path=f's3://{bucket}/probability_words'

In [ ]:
import problog
import pygrove
from problog.program import PrologString
from problog.core import ProbLog
from problog import get_evaluatable
from sorcery import dict_of
from string import Template
from tqdm.auto import tqdm

df = pd.read_csv(
    "https://raw.githubusercontent.com/wadefagen/datasets/master/Perception-of-Probability-Words/survey-results.csv"
)
prob_words = {
    k.strip('"').lower(): v / 100 for (k, v) in df.median(axis=0).to_dict().items()
}
prob_words["certain"] = 1.0
prob_words["impossible"] = 0.0


def get_prob_word(y):
    p_w = sorted(list(prob_words.items()), key=lambda x: random.random())  # break ties
    return min(p_w, key=lambda x: abs(y - x[1]))[0]


distractors = defaultdict(list)
for x, px in prob_words.items():
    for y, py in prob_words.items():
        if abs(px - py) > 0.4:
            distractors[x] += [y]


In [ ]:
df.median(axis=0)

In [ ]:
# "{expression} that {fact1}" should be grammatical
map={'about even': 'chances are about even',
 'almost certain': 'it is almost certain',
 'almost no chance': 'there is almost no chance',
 'better than even': 'there is a better than even chance',
 'chances are slight': 'chances are slight',
 'highly likely': 'it is highly likely',
 'highly unlikely': 'it is highly unlikely',
 'improbable': 'it is improbable',
 'likely': 'it is likely',
 'little chance': 'there is little chance',
 'probably': 'it is probably the case',
 'probable': 'it is probable',
 'probably not': 'it is probably not the case',
 'unlikely': 'it is unlikely',
 'very good chance': 'there is a very good chance',
 'we believe': 'we believe',
 'we doubt': 'we doubt',
 'certain': 'it is certain',
'impossible':'it is impossible'}

In [ ]:
p = PrologString("""
and(A,B) :- A,B.
or(A,B) :- A;B.
nand(A,B) :- not(and(A,B)).
nor(A,B) :- not(or(A,B)).
xor(A,B) :- or(A,B), nand(A,B).

0.3::factA.
0.3::factB.
0.3::factC.

0.3::factX:-and(A,B).
0.3::factY:-factA;factC.
0.3::factZ:-factA;factC.

0.3::conclusion:-or(factA,factC).

query(conclusion).
""")

get_evaluatable().create_from(p).evaluate()

In [ ]:
def generate_template(hop=2):

    second_hop = """%hop1
  $p4::$factX:-$op1($factX1,$factX2).
  $p5::$factY:-$op2($factY1,$factY2).
  $p6::$factZ:-$op3($factZ1,$factZ2)."""

    if hop == 1:
        second_hop = ""
    s = f"""
  and(A,B) :- A,B.
  or(A,B) :- A;B.
  nand(A,B) :- not(and(A,B)).
  nor(A,B) :- not(or(A,B)).
  xor(A,B) :- or(A,B), nand(A,B).

  %hop0
  $p1::$factA.
  $p2::$factB.
  $p3::$factC.
  {second_hop}

  %hop2
  conclusion:-$op4($factC1,$factC2).

  query(conclusion).
  """
    sub = {f"p{i}": random.random() for i in range(1, 8)}
    sub = {f"p{i}": random.choice(list(prob_words.values())) for i in range(1, 8)}
    sub = {**sub,**{f"op{i}": random.choice(["and", "or", "xor"]) for i in range(1, 5)}}

    for c in "XYZ":
        sub[f"fact{c}1"] = random.choice(["$factA", "$factB", "$factC"])
        sub[f"fact{c}2"] = random.choice(
            [fact for fact in ["$factA", "$factB", "$factC"]
                if fact != sub[f"fact{c}1"]
            ]
        )
    all_facts = ["$factA", "$factB", "$factC", "$factX", "$factY", "$factZ"]

    if hop == 1:
        all_facts = ["$factA", "$factB", "$factC"]
    sub[f"factC1"] = random.choice(all_facts)
    sub[f"factC2"] = random.choice([fact for fact in all_facts if fact != sub[f"factC1"]])

    s_ = Template(s).safe_substitute(sub).replace("$", "")
    return s_


In [ ]:
def verbalize_line(x):
    if "::" in x:
        p = float(x.split("::")[0])
        prefix = f"{map[get_prob_word(p)]} that"
    else:
        p = 1
        prefix = ""
    facts = re.findall("fact.", x)
    if ":-" in x:
        op = x.split(":-")[1].split("(")[0]
        if len(facts) == 3:
            midfix = f"{prefix} if"
            suffix = f"then {facts[0]}"
        else:
            suffix = midfix = ""
        if op == "and":
            return f"{midfix} '{facts[-2]} and {facts[-1]}' {suffix}."
        if op == "or":
            return f"{midfix} '{facts[-2]}' or '{facts[-1]}' or both {suffix}."
        if op == "xor":
            return f"{midfix} either '{facts[-2]}' or '{facts[-1]}' but not both {suffix}."
    else:
        return f"{prefix} {facts[0]}"


def score(s_):
    conclusion = problog.logic.Term("conclusion")
    return get_evaluatable().create_from(PrologString(s_)).evaluate()[conclusion]


def verbalize(s_):
    case = lambda x: x[0].upper() + x[1:]
    s = "\n".join(
        [case(verbalize_line(x).strip()) for x in s_.split("\n") if "fact" in x]
    )
    return s


In [ ]:
s_=generate_template(hop=2)
verbalize(s_),score(s_)

In [ ]:
np.mean([get_evaluatable().create_from(PrologString(generate_template())).evaluate()
[ problog.logic.Term('conclusion')] for _ in range(100)])

In [ ]:
class Babi:
    def __init__(self):
        df=pd.read_csv('https://sileod.s3.eu-west-3.amazonaws.com/babi/babi-nli.csv')
        stopwords='the to is a of and'.split()
        df=df[df.premise.map(lambda x:type(x)==str)]
        df=pd.DataFrame(sum(list(df.sample(1000).premise.map(lambda x: re.split(r'[.,]\s', x))),[]),columns=['fact'])
        df=df[~df.fact.map(lambda x: set(x.split())&{'that','and','there','afraid'}).map(bool)]
        df['concepts']=df.fact.map(lambda x: [a for a in x.split() if a.lower() not in stopwords])
        df=df[df.fact.map(len)<25]
        self.df=df

    def sample_disjoint(self,n=5):
        '''sample n unrelated facts'''
        df=self.df.copy()
        l=[]
        for _ in range(n):
            x=df.sample().iloc[0]
            l+=[x.fact if x.fact.endswith('.') else x.fact+'.']
            df=df[~df.concepts.map(lambda y:bool(set(y)&set(x.concepts)))]
        return l

In [ ]:
def get_split(x):
    if x<0.8:
        return 'train'
    if x<0.9:
        return 'validation'
    else:
        return 'test'

def punctation_fix(s):
    for x in [',.','...',', .','..']:
        s=s.replace(x,'.')
    return s.rstrip('.,')+'.'

def composition_reasoning(n=5000,hop=2):
    data = Babi()
    l = []
    for _ in tqdm(range(n)):
        s_ = generate_template(hop)
        s = verbalize(s_).replace('Fact','fact')
        probability = score(s_)

        placeholders = list(set(re.findall("fact.", s, flags=re.IGNORECASE)))
        facts = data.sample_disjoint(len(placeholders))
        for placeholder, fact in zip(placeholders, facts):
            s = s.replace(placeholder, fact.replace(".", ","))
        context = punctation_fix(". ".join(s.split("\n")[:-1]))
        hypothesis = punctation_fix(s.split("\n")[-1])


        l += [dict_of(context, hypothesis, probability)]
    df = pd.DataFrame(l)
    df["rnd"] = sorted(np.linspace(0,1,len(df)),key=lambda x:random.random())
    df['split']=df.rnd.map(get_split)
    return df


In [ ]:
def reformat(x):
    x['label']=random.randint(0,1)
    x['hypothesis_assertion']= x['hypothesis']
    x['hypothesis']=[x.invalid_hypothesis,x.valid_hypothesis][x['label']]
    return x

def verbalize_hypothesis(df):
    df['probability_word']=df.probability.map(get_prob_word)
    df['distractor']=df.probability_word.map(lambda x:random.choice(distractors[x]))
    lcase=lambda x:x[0].lower()+x[1:]
    ucase=lambda x:x[0].upper()+x[1:]

    df['valid_hypothesis'] = df.apply(lambda x: f'{ucase(map[x.probability_word])} that {lcase(x.hypothesis)}',axis=1)
    df['invalid_hypothesis'] = df.apply(lambda x: f'{ucase(map[x.distractor])} that {lcase(x.hypothesis)}',axis=1)
    df=df.apply(reformat,axis=1)
    return df

In [ ]:
df=verbalize_hypothesis(composition_reasoning(hop=1))
for split in ['train','test','validation']:
    df[df['split']==split].to_csv(f'{output_path}/reasoning_1hop_{split}.csv',index=False)

In [ ]:
df=verbalize_hypothesis(composition_reasoning(hop=2))
for split in ['train','test','validation']:
    df[df['split']==split].to_csv(f'{output_path}/reasoning_2hop_{split}.csv',index=False)

In [ ]:
for split in ['train','test','validation']:
    df=pd.read_csv(f'{usnli_path}/{split}.csv',names=['context','hypothesis','_','probability'],header=0)
    df=verbalize_hypothesis(df)
    df.to_csv(f'{output_path}/usnli_{split}.csv',index=False)

In [ ]:
df

# Noise robustness

In [ ]:
def score(T):
    return fc.first(get_evaluatable().create_from(PrologString(T)).evaluate().values())

errors=[]
noises=[0,0.1,0.2,0.3,0.4,0.5]
for noise in noises:

    l=[]
    for _ in range(500):
        T=generate_template(hop=2)
        T_=copy.deepcopy(T)
        for p in re.findall('0\.\d+',T):
            T_=T_.replace(p, str(np.clip(float(p)+np.random.randn()*noise,0,1)))
        l+=[score(T_)-score(T)]
    errors+=[np.mean(np.abs(l)>0.4)]

In [ ]:
list(zip(noises,errors))

In [ ]:
plt.plot(noises,errors)

In [ ]:
df=composition_reasoning(n=10,hop=2)
print("\n----\n".join(df.context))